In [5]:
import os

import pandas as pd
import numpy as np

from datetime import datetime, timedelta
from pytz import timezone

from src.webscraping import (
    activate_web_driver,

)

In [16]:

# initate a webdriver in selenium 
# since website data is dynamically generated
driver = activate_web_driver('firefox')


# get today's games on NBA schedule

from bs4 import BeautifulSoup as soup

NBA_SCHEDULE = "https://www.nba.com/schedule"

driver.get(NBA_SCHEDULE)

source = soup(driver.page_source, 'html.parser')


# Get how many games are scheduled for today
# The specified heading lists the number of games (e.g. "8 GAMES")
CLASS_TODAYS_HEADING = "ScheduleDay_sdWeek__iiTmo"
game_count = (source.find('h6', {'class':CLASS_TODAYS_HEADING})).text
game_count = int(list(filter(str.isdigit, game_count))[0]) #strip-out just numeric part

# Get the block of all of todays games
# All of todays games are in the specified div block
CLASS_TODAYS_GAMES = "ScheduleDay_sdGames__NGdO5"
todays_games = source.find('div', {'class':CLASS_TODAYS_GAMES})

# Get the teams playing
# Each team listed in todays block will have a href with the specified anchor class
# e.g. <a href="/team/1610612743/nuggets/" class="Anchor_anchor__cSc3P Link_styled__okbXW" ...
# href includes team ID (1610612743 in example)
# first team is visitor, second team is home
CLASS_ID = "Anchor_anchor__cSc3P Link_styled__okbXW"
links = todays_games.find_all('a', {'class':CLASS_ID})
links_list = [i.get("href") for i in links]
links_list



[WDM] - Downloading: 19.0kB [00:00, 9.78MB/s]                   


['/team/1610612743/nuggets/',
 '/team/1610612740/pelicans/',
 '/team/1610612756/suns/',
 '/team/1610612759/spurs/',
 '/team/1610612738/celtics/',
 '/team/1610612751/nets/',
 '/team/1610612763/grizzlies/',
 '/team/1610612765/pistons/',
 '/team/1610612739/cavaliers/',
 '/team/1610612752/knicks/',
 '/team/1610612747/lakers/',
 '/team/1610612764/wizards/',
 '/team/1610612741/bulls/',
 '/team/1610612758/kings/',
 '/team/1610612754/pacers/',
 '/team/1610612757/blazers/']

In [15]:
print(todays_games)

<div class="ScheduleDay_sdGames__NGdO5"><div class="ScheduleGame_sg__RmD9I"><div class="ScheduleGame_sgContent__lHCIC"><div class="ScheduleGame_sgStatus__NqtTI"><span class="ScheduleStatusText_base__Jgvjb">3:30 pm ET</span><div class="Broadcasters_base__Wet1u ScheduleGame_sgBroadcasters__O2nwT Broadcasters_vertical__cOo2a" tabindex="0"><div class="Broadcasters_section__ISlyP"><a data-content="DEN @ NOP, 2022-12-04" data-content-id="0022200344" data-content-type="packages" data-id="nba:schedule:subscribe:undefined" data-section="schedule:broadcasters" data-text="LEAGUE PASS" data-track="click" data-type="logo" href="https://www.nba.com/watch/league-pass-stream"><img alt="LEAGUE PASS" class="Broadcasters_icon__82MTV" role="presentation" src="https://cdn.nba.com/logos/nba/broadcast_logos/L/lp-hor.svg" title="LEAGUE PASS" width="100"/></a></div><div class="Broadcasters_section__ISlyP"><p class="Broadcasters_title__B1dGd">LOCAL TV:</p><p><span class="Broadcasters_tv__AIeZb" data-link-enable